In [129]:
import pandas as pd 
import csv
import numpy as np
!ls zippedData | grep csv > file_list.txt
!cat file_list.txt
# # eyes
# # heavy_check_mark
# # call_me_hand





# with open ('file_list.txt', 'r') as f:
#     csv_list= f.readlines()
# file_dict = {}
# for csv in csv_list:
#     key='ZippedData/'+csv.strip()
#     df_name=csv.strip()
#     df_name=df_name.split('.csv.gz')[0]
#     df=pd.read_csv(key, compression='gzip')
#     file_dict[df_name] = df

# print( file_dict.keys() )
# file_dict[ " <key_name> "]

bom.movie_gross.csv.gz
imdb.name.basics.csv.gz
imdb.title.akas.csv.gz
imdb.title.basics.csv.gz
imdb.title.crew.csv.gz
imdb.title.principals.csv.gz
imdb.title.ratings.csv.gz
tmdb.movies.csv.gz
tmdb_genres.csv.gz
tmdb_imdb_gross.csv.gz
tmdb_imdb_gross_full.csv
tmdb_imdb_gross_test.csv.gz
tmdb_movie_ids.csv.gz
tn.movie_budgets.csv.gz


In [80]:
with open ('file_list.txt', 'r') as f:
    csv_list= f.readlines()
file_dict = {}
for csv in csv_list:
    key='ZippedData/'+csv.strip()
    df_name=csv.strip()
    df_name=df_name.split('.csv.gz')[0]
    df=pd.read_csv(key, compression='gzip')
    file_dict[df_name] = df
file_dict.keys

BadGzipFile: Not a gzipped file (b',i')

In [81]:
rt_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz',sep = '\t', compression = 'gzip', encoding = 'latin-1' )

rt_movie_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz',sep = '\t', compression = 'gzip', encoding = 'latin-1' )



In [82]:
rt_reviews_info = rt_movie_info[['id', 'director', 'runtime', 'rating']]

rt_reviews_cleaned = rt_reviews[['id', 'rating', 'critic']]
rt_df = rt_reviews_info.merge(rt_reviews_cleaned, how = 'inner', on = 'id')

rt_df.head(1)

,id,director,runtime,rating_x,rating_y,critic
0,3,David Cronenberg,108 minutes,R,3/5,PJ Nabarro


In [83]:
# NORMALIZE RATINGS RATING>FLOAT

def norm_ratings(rating):
    grades = {"A":10,"B":8,"C":6,"D":4,"F":2,"A+":10,"B+":9,"C+":7,"D+":5,\
              "F+":3,"A-":9,"B-":7,"C-":5,"D-":3,"F-":0, '3 1/2': 3.5, 'T': None, 'N':None, 'R':None}
    
    if rating in grades.keys():
        norm=grades[rating]
        return norm
    elif ('/') in rating:
        r = rating.split('/')
        norm = float(r[0])/float(r[1])*10
        return norm
    elif ('-') in rating and len(rating) > 2:
        r = rating.split('-')
        norm = float(r[0])/float(r[1])*10
        return norm
    else:
        norm = float(rating)
        return norm

    
    
rt_df.dropna(subset=['rating_y'],inplace=True)    
rt_df['critic_rating'] = rt_df['rating_y'].apply(norm_ratings)

In [84]:
# CLEAN RUNTIMES FLOAT>INT

def clean_runtime(runtime):
    minutes = runtime.split()[0]
    return int(minutes) 


rt_df.dropna(subset=['runtime'],inplace=True)    
rt_df['runtime'] = rt_df['runtime'].apply(clean_runtime)
rt_df.drop(columns = 'rating_y', inplace = True)
rt_df.head()

,id,director,runtime,rating_x,critic,critic_rating
0,3,David Cronenberg,108,R,PJ Nabarro,6.0
6,3,David Cronenberg,108,R,Eric D. Snider,6.0
7,3,David Cronenberg,108,R,Matt Kelemen,4.0
11,3,David Cronenberg,108,R,Emanuel Levy,7.0
12,3,David Cronenberg,108,R,Christian Toto,5.0


In [85]:
# ADDING DIRECTOR NAME ID TO RT_DF

name_key = file_dict['imdb.name.basics'][['nconst', 'primary_name']]




rt_df2 = rt_df.merge(name_key, left_on = 'director', right_on = 'primary_name')
rt_df2 = rt_df2.drop(columns = 'primary_name')

rt_df2.head()

,id,director,runtime,rating_x,critic,critic_rating,nconst
0,3,David Cronenberg,108,R,PJ Nabarro,6.0,nm0000343
1,3,David Cronenberg,108,R,Eric D. Snider,6.0,nm0000343
2,3,David Cronenberg,108,R,Matt Kelemen,4.0,nm0000343
3,3,David Cronenberg,108,R,Emanuel Levy,7.0,nm0000343
4,3,David Cronenberg,108,R,Christian Toto,5.0,nm0000343


In [86]:
movie_director_code = file_dict['imdb.title.crew'][['tconst', 'directors']]
movie_director_code['director'] = movie_director_code['directors'].str.split(',')
movie_director_code.drop(columns = 'directors', inplace = True)
movie_director_code = movie_director_code.explode('director')



In [87]:
movie_id_names_time = file_dict['imdb.title.basics'][['tconst', 'primary_title', 'original_title', 'runtime_minutes']]
half_way_clean = movie_id_names_time.merge(movie_director_code, how = 'inner', on = 'tconst')
half_way_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169262 entries, 0 to 169261
Data columns (total 5 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   tconst           169262 non-null  object 
 1   primary_title    169262 non-null  object 
 2   original_title   169240 non-null  object 
 3   runtime_minutes  133324 non-null  float64
 4   director         163535 non-null  object 
dtypes: float64(1), object(4)
memory usage: 7.7+ MB


In [88]:
def float_conv(flt):
    integer = int(flt)
    return integer


half_way_clean.dropna(subset = ['runtime_minutes'], inplace = True)
half_way_clean['runtime_minutes'] = half_way_clean['runtime_minutes'].apply(float_conv)

In [89]:
# half_way_clean.head(1)
rt_df.head(1)

,id,director,runtime,rating_x,critic,critic_rating
0,3,David Cronenberg,108,R,PJ Nabarro,6.0


In [90]:
combined_df = rt_df2.merge(half_way_clean, left_on = ['runtime', 'nconst'], right_on = ['runtime_minutes', 'director'])

len(combined_df['id'].unique())
combined_df.head()

,id,director_x,runtime,rating_x,critic,critic_rating,nconst,tconst,primary_title,original_title,runtime_minutes,director_y
0,14,Taylor Hackford,117,R,Laura Hiros,6.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
1,14,Taylor Hackford,117,R,James Luxford,4.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
2,14,Taylor Hackford,117,R,Brian Orndorf,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
3,14,Taylor Hackford,117,R,Nick Schager,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
4,14,Taylor Hackford,117,R,Frank Swietek,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431


In [91]:
tester = combined_df.rename({'director_x': 'DirectorName', 'rating_x': "MPArating", 'nconst':'DirectorId'}, inplace = True)

combined_df
# tester.head()

,id,director_x,runtime,rating_x,critic,critic_rating,nconst,tconst,primary_title,original_title,runtime_minutes,director_y
0,14,Taylor Hackford,117,R,Laura Hiros,6.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
1,14,Taylor Hackford,117,R,James Luxford,4.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
2,14,Taylor Hackford,117,R,Brian Orndorf,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
3,14,Taylor Hackford,117,R,Nick Schager,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
4,14,Taylor Hackford,117,R,Frank Swietek,5.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431
...,...,...,...,...,...,...,...,...,...,...,...,...
6697,1944,Michel Gondry,119,PG-13,Edward Douglas,8.5,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273
6698,1944,Michel Gondry,119,PG-13,Matt Singer,6.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273
6699,1944,Michel Gondry,119,PG-13,Perri Nemiroff,6.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273
6700,1944,Michel Gondry,119,PG-13,Sam Bathe,6.0,nm0327273,tt0990407,The Green Hornet,The Green Hornet,119,nm0327273


In [102]:
tester = combined_df.merge(file_dict['imdb.title.ratings'], on = 'tconst')
# tester2 = head()
# len(tester['id'].unique())
# tester.info()
tester.head(1)

,id,director_x,runtime,rating_x,critic,critic_rating,nconst,tconst,primary_title,original_title,runtime_minutes,director_y,averagerating,numvotes
0,14,Taylor Hackford,117,R,Laura Hiros,6.0,nm0000431,tt1125929,Love Ranch,Love Ranch,117,nm0000431,5.6,2742


In [103]:
with open ('file_list.txt', 'r') as f:
    file_list= f.readlines()

In [111]:
# csv_dict = {}

# for csv in file_list:
#     file='ZippedData/'+csv.strip()
#     df_name=csv.strip()
#     if 'csv' in file:
#         df_name=df_name.split('.csv.gz')[0]
#         df=pd.read_csv(file, compression='gzip') ####
#     elif 'tsv' in file:
#         df_name=df_name.split('.tsv.gz')[0]
#         df=pd.read_csv(file, compression='gzip', delimiter='\t', encoding='latin-1')
#     else: print(file, ": unkown file!")
#     csv_dict[df_name] = df



In [136]:

# csv_dict.get('tmdb_imdb_gross_test.csv.gz')
# df5 = df.drop(columns = ['Unnamed: 0', 'Unnamed: 0.1'])
# df5.head()
# df6 = df.replace(0, np.nan, inplace=True)
# df6.head()
df.dropna()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25025 entries, 0 to 25024
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            25025 non-null  int64  
 1   tconst        23609 non-null  object 
 2   release_date  25021 non-null  object 
 3   revenue       3215 non-null   float64
 4   budget        4722 non-null   float64
dtypes: float64(2), int64(1), object(2)
memory usage: 1.1+ MB


In [107]:
csv_dict.get('tmdb_imdb_gross_full')

df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [ ]:
money_cols = full_gross[['revenue', 'budget']]
money_cols.info()

In [67]:
tester.merge(money_cols,left_on = 'tconst', right_on = 'tconst')

KeyError: 'tconst'